# Re-evaluation of magopter's sweep partitioning

Having another look at the sweep partitioning section of homogenise() to see if it can be sped up. 

In [1]:
%matplotlib tk
import scipy.signal as sig
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import sys 
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.magnum.magoptoffline as mg
import analysis.scripts.magnum.magnum_advanced_analysis as maa
import flopter.core.fitters as fts
import flopter.core.constants as c
import flopter.core.ivdata as iv

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [7]:
# shot_info = maa.get_shot_info_for_analysis(157)
shot_info = maa.get_shot_info_for_analysis(283)
shot_info

('2019-06-05_Leland/',
 '2019-06-05 11h 17m 10s TT_06698973108763191132.adc',
 'a283_276_6698973108763191296',
 array(2.36582169),
 array(1.19102882e+19),
 ('S', 'L'),
 array(1.),
 100,
 (10.04, 10.04),
 (1, 1000))

In [4]:
# maa.shot_dataset.sel(shot_number=157)

In [8]:
folder, adc_file, output_tag, ts_temp, ts_dens, probe_designations, shunt_resistance, downsampling_factor, cabling_resistance, sweep_range = shot_info

In [24]:
# adc_freqs is not actually the sweep frequency, just the sampling rate

# frequency = maa.shot_dataset.sel(shot_number=157)#['adc_freqs'].values
# frequency

<xarray.Dataset>
Dimensions:                 ()
Coordinates:
    shot_number             int32 157
    ts_number               float64 ...
    ts_timestamp            float64 ...
    ts_time                 datetime64[ns] ...
    adc_index               float64 162.0
    adc_time                datetime64[ns] ...
Data variables:
    adc_filename            object '2019-06-03 16h 02m 56s TT_06698304580043636299.adc'
    ts_density              float64 nan
    ts_temperature          float64 nan
    ts_d_density            float64 nan
    ts_d_temperature        float64 nan
    adc_folder              object '2019-06-03_Leland/'
    adc_calibration_index   object '120'
    adc_4_probe             object 'S'
    adc_5_probe             object 'L'
    ts_temp_max             float64 nan
    ts_dens_max             float64 nan
    shot_end_time           datetime64[ns] 2019-06-03T15:02:57
    ts_fl                   bool False
    shot_time               datetime64[ns] 2019-06-03T15:02:42
 

In [10]:
mg.Magoptoffline._FOLDER_STRUCTURE = 'data/external/magnum/'
print('"{}" \t\t "{}"'.format(folder, adc_file))

# Create magopter object
print('Creating magopter object')
magopter = mg.Magoptoffline(folder, adc_file, shunt_resistor=shunt_resistance, cabling_resistance=cabling_resistance)
magopter._VOLTAGE_CHANNEL = 3
magopter._PROBE_CHANNEL_3 = 4
magopter._PROBE_CHANNEL_4 = 5
# magopter.prepare(down_sampling_rate=downsampling_factor, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=45000, crit_ampl=None, homogenise_fl=False)
magopter.prepare(down_sampling_rate=downsampling_factor, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None, homogenise_fl=False)

"2019-06-05_Leland/" 		 "2019-06-05 11h 17m 10s TT_06698973108763191132.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.


In [11]:
fig, ax = plt.subplots()

print(f'raw_time: {len(magopter.raw_time)}, raw_voltage: {len(magopter.raw_voltage)}')

# make a slice of interest
slc_oi = slice(0, int(0.05 * len(magopter.raw_time)))
ax.plot(magopter.raw_time[slc_oi], magopter.raw_voltage[slc_oi])
ax.plot(magopter.raw_time[slc_oi], magopter.voltage[0][slc_oi])
ax.plot(magopter.raw_time[slc_oi], magopter.voltage[1][slc_oi])

raw_time: 100000, raw_voltage: 100000


In [14]:
fig, ax = plt.subplots()

ax.plot(magopter.raw_voltage[slc_oi], magopter.raw_current[0][slc_oi])
ax.plot(magopter.voltage[0][slc_oi], magopter.current[0][slc_oi])
ax.plot(magopter.voltage[0][slc_oi], magopter.current[1][slc_oi])

In [13]:
print(600000 / 10000000)

0.06


In [8]:
# baseline case, as it is in magoptoffline

import time

triangle = fts.TriangleWaveFitter()
frequency = None

start = time.time()
if frequency is None:
    # Use fourier decomposition from get_frequency method in triangle fitter to get frequency if none given
    frequency = triangle.get_frequency(magopter.raw_time, magopter.raw_voltage, accepted_freqs=magopter._ACCEPTED_FREQS)
end_freq = time.time()

# Smooth the voltage to get a first read of the peaks on the triangle wave
smoothed_voltage = sig.savgol_filter(magopter.raw_voltage, 21, 2)
end_smooth = time.time()

top = sig.argrelmax(smoothed_voltage, order=100)[0]
end_argrelmax = time.time()

bottom = sig.argrelmin(smoothed_voltage, order=100)[0]
end_argrelmin = time.time()

_peaks = magopter.raw_time[np.concatenate([top, bottom])]
_peaks.sort()

end_sort = time.time()

# Get distances between the peaks and filter based on the found frequency
_peak_distances = np.diff(_peaks)
threshold = (1 / (2 * frequency)) - 0.001
_peaks_ind = np.where(_peak_distances > threshold)[0]
end_where = time.time()

# Starting from the first filtered peak, arrange a period-spaced array
peaks_refined = np.arange(_peaks[_peaks_ind[0]], magopter.raw_time[-1], 1 / (2 * frequency))
magopter.peaks = peaks_refined
end_arange = time.time()

print(
    f'to freq: {end_freq - start}\n'
    f'to smooth: {end_smooth - end_freq}\n'
    f'to a-r-max: {end_argrelmax - end_smooth}\n'
    f'to a-r-min: {end_argrelmin - end_argrelmax}\n'
    f'to sort: {end_sort - end_argrelmin}\n'
    f'to where: {end_where - end_sort}\n'
    f'to arange: {end_arange - end_where}\n'
)

to freq: 1.531310796737671
to smooth: 0.2207503318786621
to a-r-max: 21.501292943954468
to a-r-min: 16.88404130935669
to sort: 0.0032448768615722656
to where: 0.0006380081176757812
to arange: 0.00017142295837402344



In [11]:
# Sped up case by taking a slice 

import time

triangle = fts.TriangleWaveFitter()
frequency = None

start = time.time()
if frequency is None:
    # Use fourier decomposition from get_frequency method in triangle fitter to get frequency if none given
    frequency = triangle.get_frequency(magopter.raw_time, magopter.raw_voltage, accepted_freqs=magopter._ACCEPTED_FREQS)
end_freq = time.time()

# Smooth the voltage to get a first read of the peaks on the triangle wave
smoothed_voltage = sig.savgol_filter(magopter.raw_voltage, 21, 2)
end_smooth = time.time()

top = sig.argrelmax(smoothed_voltage[slc_oi], order=100)[0]
end_argrelmax = time.time()

bottom = sig.argrelmin(smoothed_voltage[slc_oi], order=100)[0]
end_argrelmin = time.time()

_peaks = magopter.raw_time[np.concatenate([top, bottom])]
_peaks.sort()
end_sort = time.time()

# Get distances between the peaks and filter based on the found frequency
_peak_distances = np.diff(_peaks)
threshold = (1 / (2 * frequency)) - 0.001
_peaks_ind = np.where(_peak_distances > threshold)[0]
end_where = time.time()

# Starting from the first filtered peak, arrange a period-spaced array
peaks_refined = np.arange(_peaks[_peaks_ind[0]], magopter.raw_time[-1], 1 / (2 * frequency))
magopter.peaks = peaks_refined
end_arange = time.time()

print(
    f'to freq: {end_freq - start}\n'
    f'to smooth: {end_smooth - end_freq}\n'
    f'to a-r-max: {end_argrelmax - end_smooth}\n'
    f'to a-r-min: {end_argrelmin - end_argrelmax}\n'
    f'to sort: {end_sort - end_argrelmin}\n'
    f'to where: {end_where - end_sort}\n'
    f'to arange: {end_arange - end_where}\n'
)

to freq: 1.7650878429412842
to smooth: 0.22937417030334473
to a-r-max: 1.1273033618927002
to a-r-min: 1.0344195365905762
to sort: 0.000156402587890625
to where: 0.00018358230590820312
to arange: 0.00012540817260742188



In [12]:
print(frequency)

plt.figure()
plt.plot(magopter.raw_time, magopter.raw_voltage)
# plt.plot(magopter.raw_time[slc_oi], triangle.fit(magopter.raw_time[slc_oi], magopter.raw_voltage[slc_oi]).fit_y)
for peak in magopter.peaks[-60:]:
    plt.axvline(x=peak, linestyle='dashed', linewidth=0.5, color='r')

100.0


## Comparison of different filter frequencies
Need to choose between cutting off the peaks in teh fourier decomposition (at 51kHz) or going for a harder blanket suppression of noise at 15kHz which would remove the noise (which may be present in the fourier decomposition) from the power supplies at ~20kHz. My thinking is that the signal at 51kHz is a harmonic of the power supply noise - but the filter at 15 looks like it's having a sizable effect and I'm not entirely sure it's justified. 

In [6]:
mg.Magoptoffline._FOLDER_STRUCTURE = '/data/external/magnum/'
print('"{}" \t\t "{}"'.format(folder, adc_file))

# Create magopter object
print('Creating magopter object')
magopter_15 = mg.Magoptoffline(folder, adc_file, shunt_resistor=shunt_resistance, cabling_resistance=cabling_resistance)
magopter_15._VOLTAGE_CHANNEL = 3
magopter_15._PROBE_CHANNEL_3 = 4
magopter_15._PROBE_CHANNEL_4 = 5
magopter_15.prepare(down_sampling_rate=downsampling_factor, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=15000, crit_ampl=None, homogenise_fl=False)

"2019-06-03_Leland/" 		 "2019-06-03 16h 02m 56s TT_06698304580043636299.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.


In [7]:
mg.Magoptoffline._FOLDER_STRUCTURE = '/data/external/magnum/'
print('"{}" \t\t "{}"'.format(folder, adc_file))

# Create magopter object
print('Creating magopter object')
magopter_nf = mg.Magoptoffline(folder, adc_file, shunt_resistor=shunt_resistance, cabling_resistance=cabling_resistance)
magopter_nf._VOLTAGE_CHANNEL = 3
magopter_nf._PROBE_CHANNEL_3 = 4
magopter_nf._PROBE_CHANNEL_4 = 5
magopter_nf.prepare(down_sampling_rate=downsampling_factor, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None, homogenise_fl=False)

"2019-06-03_Leland/" 		 "2019-06-03 16h 02m 56s TT_06698304580043636299.adc"
Creating magopter object
Running an offline magopter object, operating in offline mode.


In [8]:
fig, ax = plt.subplots(2, 3)

# ax[0][0].plot(magopter.raw_voltage[slc_oi], magopter_15.raw_current[0][slc_oi])
# ax[0][0].plot(magopter_nf.voltage[0][slc_oi], magopter_nf.current[0][slc_oi])
ax[0][0].plot(magopter.voltage[0][slc_oi], magopter.current[0][slc_oi])
ax[0][0].plot(magopter_15.voltage[0][slc_oi], magopter_15.current[0][slc_oi])

# ax[1][0].plot(magopter_15.raw_voltage[slc_oi], magopter_15.raw_current[0][slc_oi])
# ax[1][0].plot(magopter_nf.voltage[1][slc_oi], magopter_nf.current[1][slc_oi])
ax[1][0].plot(magopter.voltage[1][slc_oi], magopter.current[1][slc_oi])
ax[1][0].plot(magopter_15.voltage[1][slc_oi], magopter_15.current[1][slc_oi])


ax[0][1].plot(magopter_15.voltage[0][slc_oi] - magopter.voltage[0][slc_oi])

ax[1][1].plot(magopter_15.voltage[1][slc_oi] - magopter.voltage[1][slc_oi])

ax[0][2].plot(magopter_15.current[0][slc_oi] - magopter.current[0][slc_oi])

ax[1][2].plot(magopter_15.current[1][slc_oi] - magopter.current[1][slc_oi])


In [9]:
# Plot for side-by-side comparison of the IV characteristics of the first 5% of the signal

fig, ax = plt.subplots(2, 2, sharex='col')

ax[0][0].plot(magopter_nf.raw_time[slc_oi], magopter_nf.current[0][slc_oi])
ax[0][0].plot(magopter.raw_time[slc_oi], magopter.current[0][slc_oi])
ax[0][0].plot(magopter_15.raw_time[slc_oi], magopter_15.current[0][slc_oi])

# ax[0][0].plot(magopter.raw_voltage[slc_oi], magopter_15.raw_current[0][slc_oi])
ax[1][0].plot(magopter_nf.raw_time[slc_oi], magopter_nf.voltage[0][slc_oi])
ax[1][0].plot(magopter.raw_time[slc_oi], magopter.voltage[0][slc_oi])
ax[1][0].plot(magopter_15.raw_time[slc_oi], magopter_15.voltage[0][slc_oi])

ax[0][1].plot(magopter_nf.voltage[0][slc_oi], magopter_nf.current[0][slc_oi])
ax[0][1].plot(magopter.voltage[0][slc_oi], magopter.current[0][slc_oi])
ax[0][1].plot(magopter_15.voltage[0][slc_oi], magopter_15.current[0][slc_oi])

ax[1][1].plot(magopter_nf.voltage[1][slc_oi], magopter_nf.current[1][slc_oi])
ax[1][1].plot(magopter.voltage[1][slc_oi], magopter.current[1][slc_oi])
ax[1][1].plot(magopter_15.voltage[1][slc_oi], magopter_15.current[1][slc_oi])

From the above plot, I'm going to go with a 45kHz filter to get rid of the large harmonics